In [8]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [9]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [10]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [11]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [12]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [13]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [14]:
## 
n='Input_A6_024' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A6_024
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.003013906858915423
test_rmse: 0.0030432632126253262
test_r2: 0.9407502866303064
feature number: 3
train_rmse: 0.002936537215829018
test_rmse: 0.0030670240095864625
test_r2: 0.9409285836880106
feature number: 4
train_rmse: 0.0024842079978108604
test_rmse: 0.0029344304259889486
test_r2: 0.9468793115236087
feature number: 5
train_rmse: 0.0024720743254105534
test_rmse: 0.0029629657282896136
test_r2: 0.9461313716234795
feature number: 6
train_rmse: 0.0024642165567532624
test_rmse: 0.002965152907631811
test_r2: 0.9461414601234763
feature number: 7
train_rmse: 0.0023932372896607094
test_rmse: 0.002902466068294067
test_r2: 0.9479807856847199
feature number: 8
train_rmse: 0.002379564041432359
test_rmse: 0.003075669786845232
test_r2: 0.9418034900708914
feature number: 9
train_rmse: 0.0023770989483929087
test_rmse: 0.00308596474941189
test_r2: 0.9414905438092637
feature number: 10
train_rmse: 0.00233793594210

train_rmse: 0.002245035944427305
test_rmse: 0.0036713822306884197
test_r2: 0.9108243595783018
feature number: 14
train_rmse: 0.0020311652715483005
test_rmse: 0.0032842106289582853
test_r2: 0.931954170179116
feature number: 15
train_rmse: 0.0018338224176942496
test_rmse: 0.003086749702970675
test_r2: 0.9410207490611029
feature number: 16
train_rmse: 0.0018318119933271502
test_rmse: 0.003094867412320762
test_r2: 0.9408273660468381
feature number: 17
train_rmse: 0.0018298099829963678
test_rmse: 0.0030997690215639013
test_r2: 0.9407074629776744
feature number: 18
train_rmse: 0.001824894737825273
test_rmse: 0.003105028354166985
test_r2: 0.9405837783322403
feature number: 19
train_rmse: 0.0018204597411156163
test_rmse: 0.00310607385851663
test_r2: 0.9404066913851414
feature number: 20
train_rmse: 0.0017909740413275422
test_rmse: 0.003254641784804775
test_r2: 0.933846078751279
Test_rmse_min = 1.fea_num: 7  2.rmse: 0.002902466068294067
Test_r2_max = 1. fea_num: 7  2.r2: 0.9479807856847199
./Da

train_rmse: 0.0024842079978108604
test_rmse: 0.0029344304259889486
test_r2: 0.9468793115236087
feature number: 5
train_rmse: 0.0024720743254105534
test_rmse: 0.0029629657282896136
test_r2: 0.9461313716234795
feature number: 6
train_rmse: 0.0024642165567532624
test_rmse: 0.002965152907631811
test_r2: 0.9461414601234763
feature number: 7
train_rmse: 0.0023932372896607094
test_rmse: 0.002902466068294067
test_r2: 0.9479807856847199
feature number: 8
train_rmse: 0.002379564041432359
test_rmse: 0.003075669786845232
test_r2: 0.9418034900708914
feature number: 9
train_rmse: 0.0023770989483929087
test_rmse: 0.00308596474941189
test_r2: 0.9414905438092637
feature number: 10
train_rmse: 0.002337935942108291
test_rmse: 0.003377499975213053
test_r2: 0.929774997667978
feature number: 11
train_rmse: 0.0023357247216885093
test_rmse: 0.00338023609968926
test_r2: 0.9296412036069006
feature number: 12
train_rmse: 0.0023154945909019354
test_rmse: 0.0034505496773767986
test_r2: 0.9261554887294026
feature n

# 可以修改第一層的神經元數量

In [15]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [16]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.01539628502598624
[0.02169617680553141, 0.005874077103606192, 0.06195722070622587, 0.0051934742710494815, 0.006525400941257158, 0.008899800690224625, 0.005321629822633552, 0.013217824898537804, 0.010804262065627327, 0.014472982955168984]


test_r2_ave: -1.9952160417569693
[-2.2629056883388152, 0.7786996977570138, -21.48878080874692, 0.8646418911103447, 0.8050217722863442, 0.6231926054407366, 0.7716149679907838, -0.3301482065524999, 0.29912120324042046, -0.012617851757102061]


feature number: 5
test_rmse_ave: 0.015156447125494115
[0.06009072507362295, 0.009185075869739448, 0.014636154216678077, 0.0089589815474337, 0.01078655543912688, 0.009699610122549227, 0.0037377112452011184, 0.005512941922926978, 0.012459004690879653, 0.016497711126783138]


test_r2_ave: -2.0800577440692694
[-24.029546860066937, 0.4589116076033215, -0.2549777023511228, 0.597203587874271, 0.46723304982464, 0.5524233601996591, 0.8873347955443315, 0.7686087781801236, 0.06799251360363

In [17]:
fea_num=[4,5,6,7]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.011009923353969456
[0.003563004329509564, 0.027853136999609698, 0.006039862479827386, 0.0026980541560274974, 0.0031426298165531602, 0.009858849664994798, 0.010347829562483181, 0.016125320427490326, 0.007297124960644658, 0.02317342114255427]


test_r2_ave: -0.1580481105774964
[0.9120024058914852, -3.9756629313131304, 0.7862844332871707, 0.9634683208506755, 0.9547771249387386, 0.5376069010552006, 0.1364711560145092, -0.9796880027303603, 0.6802899642357675, -1.5960304780050203]


feature number: 5
test_rmse_ave: 0.01414776456965163
[0.03902383815790143, 0.013380807859550297, 0.01799136698932384, 0.007705089673937303, 0.00497871075600513, 0.005989006917793726, 0.00857768098121411, 0.030085831394710376, 0.008296602707820603, 0.0054487102582594935]


test_r2_ave: -1.2224204545995707
[-9.555967928155344, -0.14833179708585686, -0.8963145649309001, 0.7020635049890058, 0.8864974788518739, 0.8293648554167307, 0.4066402337461108, -5.89134637738286, 0.586711529459

In [18]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.009830347341164343
[0.0042752996033838505, 0.01721528346081188, 0.005608440643510616, 0.007529643122670174, 0.01732858230893686, 0.0040937490403919995, 0.00293366080486196, 0.007840454743000113, 0.02630873112017618, 0.005169628563899798]


test_r2_ave: 0.12266170294668317
[0.8733015775589177, -0.9007766030325981, 0.8157250458946309, 0.7154771863801783, -0.3749844385611045, 0.9202737593126398, 0.9305938248549732, 0.5319820077261319, -3.1557796010945474, 0.8708042704276099]


feature number: 5
test_rmse_ave: 0.014545795685843615
[0.003788827624098633, 0.013215180387441594, 0.006132990026721912, 0.006225618371450708, 0.009018933991166446, 0.026725943840709403, 0.020003021416977636, 0.02197237820936411, 0.03329605140252486, 0.005079011587980839]


test_r2_ave: -0.9088441344354765
[0.9004943381678439, -0.1200796650469147, 0.7796431412331837, 0.8054938671404871, 0.6275378612733521, -2.3980123917146496, -2.2267811758715, -2.675650218239838, -5.65638094730784

In [ ]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.014772542394221589
[0.004438305121244488, 0.0058563563040851885, 0.027421589769091394, 0.023625854402545797, 0.05324406014700045, 0.00365909629250726, 0.005009786722423425, 0.014748658327922968, 0.006040015560818677, 0.0036817012945762457]


test_r2_ave: -1.2750862558123237
[0.8634560666128184, 0.7800329124529427, -3.4052170836624285, -1.8011950856477204, -11.981176255110057, 0.9363048254160992, 0.7975970725470157, -0.6560943445022938, 0.7809572833265953, 0.9344720504437909]


feature number: 5
test_rmse_ave: 0.012079845497080823
[0.004348403390298381, 0.011096421983468097, 0.00834851808257965, 0.016073102079201037, 0.003609335215462562, 0.012059196771058926, 0.004180881912026717, 0.02624325903896921, 0.02219252228772946, 0.012646814210014177]


test_r2_ave: -0.24917592990708934
[0.8689316748913334, 0.2102881595648548, 0.591679823698851, -0.29648565259856774, 0.940347842801483, 0.3081760086026425, 0.8590340841749018, -4.243431262326835, -1.95710013835

In [ ]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')